In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def sliding_window(df,window_size,first_candle): 

    df = df.sort_index(ascending=False)
    df = df.iloc[first_candle: first_candle + window_size]
    return df


In [3]:
#Preparação dos dados:
df = pd.read_csv('dados_eurusd_1_12_2023.csv')
print(df.describe())
print(df.isnull().sum())

               open          high           low         close
count  74005.000000  74005.000000  74005.000000  74005.000000
mean       1.073304      1.073648      1.072940      1.073269
std        0.043913      0.043857      0.043959      0.043897
min        0.953890      0.955150      0.953550      0.953880
25%        1.053130      1.053540      1.052800      1.053210
50%        1.074720      1.074940      1.074510      1.074790
75%        1.099290      1.099510      1.098960      1.099280
max        1.168880      1.169190      1.168610      1.168880
time     0
open     0
high     0
low      0
close    0
dtype: int64


In [4]:
saldo = 10000

In [ ]:
df = pd.read_csv('dados_eurusd_1_12_2023.csv')
df['action'] = np.random.choice(['buy', 'sell', 'hold'], size=len(df))
df['profit'] = np.random.uniform(-1, 1, len(df))

# Divida os dados em treinamento e teste
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)

# Configurações de tamanho
window_size = 8600  # Tamanho da janela de dados
batch_size = 16  # Tamanho do lote


In [ ]:
#Nesse quadrado, fazer o modelo

# Prepare os dados para treinamento
def prepare_data(df, window_size):
    data = []
    actions = []
    profits = []
    for i in range(len(df) - window_size):
        window = df.iloc[i:i + window_size][['open', 'high', 'close', 'low']].values
        action = df.iloc[i + window_size]['action']
        profit = df.iloc[i + window_size]['profit']
        data.append(window)
        actions.append(action)
        profits.append(profit)
    return np.array(data), np.array(actions), np.array(profits)

X_train, y_train, profits_train = prepare_data(train_data, window_size)
X_test, y_test, profits_test = prepare_data(test_data, window_size)

# Converta rótulos para representação one-hot
y_train_onehot = pd.get_dummies(y_train).values
y_test_onehot = pd.get_dummies(y_test).values

# Define the CNN-GRU model architecture (example from a previous response)
model = tf.keras.Sequential([
    layers.Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(window_size, 1)),
    layers.MaxPooling1D(pool_size=2),
    layers.GRU(units=64, return_sequences=True),
    layers.GRU(units=32),
    layers.Dense(3, activation='softmax')  # Assuming 3 output classes
])

# Compile the model with an optimizer that handles backpropagation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Example optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#Nesse quadrado: calcular cada epoch de uma vez

import numpy as np

# Set the number of epochs
num_epochs = 10

# Train the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model.fit(candles, targets, epochs=1, batch_size=32)

    # Perform the trading actions and calculate the profit/loss after each epoch
    predictions = model.predict(candles)
    entry_prices = candles[:, :, 3]  # Close price of each candle
    stop_losses = entry_prices * 0.99  # Stop loss at 1%

    profit_loss = np.zeros(batch_size)

    for i in range(batch_size):
        action = np.argmax(predictions[i])
        if action == 0:  # Buy
            profit_loss[i] = entry_prices[i][-1] - stop_losses[i][-1]
        elif action == 1:  # Sell
            profit_loss[i] = stop_losses[i][-1] - entry_prices[i][-1]

    # Perform backpropagation using the profit/loss after each epoch
    # Modify the model's loss function to include the profit/loss
model.compile(optimizer='adam', loss=lambda y_true, y_pred: profit_loss(y_true, y_pred), metrics=['accuracy'])

# Train the model
model.fit(candles, targets, epochs=10, batch_size=32)